In [22]:
# ! pip3 install python-docx pdfplumber pandas openpyxl

In [23]:
# # 关键词词典

# INVEST_HUMAN = {
#     '教育': ['义务教育', '高中教育', '职业教育', '职教', '高等教育', '教师培训', '学生资助',
#             '课后服务', '产教融合', '教育信息化', '特岗教师', '营养午餐', '学位供给'],
#     '卫生健康': ['公共卫生', '疫情防控', '基层医疗', '医保', '妇幼保健', '精神卫生',
#                 '家庭医生', '健康促进', '中医药', '疾控', '医疗能力提升'],
#     '社会保障': ['养老金', '低保', '特困供养', '残疾人', '托育', '儿童福利', '就业培训',
#                 '创业扶持', '公益性岗位', '稳岗补贴', '技能培训'],
#     '人才发展': ['人才引进', '安家费', '租房补贴', '科研启动', '高层次人才', '青年发展',
#                 '技能人才', '工匠'],
#     '文化治理': ['公共文化', '社区治理', '法律援助', '志愿服务', '数字素养']
# }

# INVEST_PHYSICAL = {
#     '基础设施': ['交通', '道路', '桥梁', '轨道交通', '地铁', '机场', '港口', '水利', '电网',
#               '5G', '数据中心', '新基建', '智慧城市', '地下管廊', '停车场', '充电桩'],
#     '产业园区': ['产业园', '开发区', '高新区', '工业园', '标准厂房', '物流基地', '冷链',
#               '孵化器', '平台公司', '土地收储'],
#     '公共建筑': ['新建.*学校', '新建.*医院', '文体场馆', '公园', '城市更新', '棚改',
#               '安置房', '保障房建设']  # 注意：用正则匹配
# }

# # 扁平化关键词列表（用于快速匹配）
# HUMAN_KEYWORDS = [kw for sublist in INVEST_HUMAN.values() for kw in sublist]
# PHYSICAL_KEYWORDS = [kw for sublist in INVEST_PHYSICAL.values() for kw in sublist]

# # 编译正则表达式（支持模糊匹配，如“新建XX学校”）
# import re
# PHYSICAL_PATTERNS = [re.compile(kw) if '.*' in kw else re.compile(re.escape(kw)) for kw in PHYSICAL_KEYWORDS]

In [24]:
# # analyze_reports_all_formats.py

# import os
# import re
# import pandas as pd
# import warnings
# import textract  # 用于 .doc / .docx / .pdf（备用）
# import pdfplumber  # 仍优先用 pdfplumber 解析 PDF（更精准）

# # 忽略 pdfplumber 警告
# warnings.filterwarnings("ignore", message="Cannot render horizontal string")

# # ================== 配置 ==================
# BASE_DIR = "./reports"
# OUTPUT_CSV = "./output/investment_classification.csv"

# def get_region_label(name):
#     name = name.strip()
#     if "东部" in name: return "东部"
#     if "中部" in name: return "中部"
#     if "西部" in name: return "西部"
#     return "未知区域"

# # 关键词（同前）
# HUMAN_KEYWORDS = [
#     '义务教育', '职业教育', '教师培训', '学生资助', '课后服务', '产教融合',
#     '公共卫生', '医保', '妇幼保健', '家庭医生', '健康促进',
#     '养老金', '低保', '托育', '就业培训', '创业扶持', '技能培训',
#     '人才引进', '安家费', '租房补贴', '高层次人才', '青年发展',
#     '公共文化', '社区治理'
# ]

# PHYSICAL_PATTERNS = []
# for kw in [
#     '交通', '道路', '桥梁', '轨道交通', '地铁', '机场', '港口', '水利', '电网',
#     '5G', '数据中心', '新基建', '智慧城市', '地下管廊', '停车场',
#     '产业园', '开发区', '高新区', '标准厂房', '物流基地',
#     r'新建.*学校', r'新建.*医院', r'城市更新', r'棚改', r'保障房'
# ]:
#     if '.*' in kw:
#         PHYSICAL_PATTERNS.append(re.compile(kw))
#     else:
#         PHYSICAL_PATTERNS.append(re.compile(re.escape(kw)))

# YEAR_PATTERN = re.compile(r'(202[1-5])')

# # ================== 文本提取函数 ==================

# def extract_text_textract(filepath):
#     """通用文本提取（支持 .doc, .docx, .pdf）"""
#     try:
#         # textract 自动根据扩展名选择解析器
#         text = textract.process(filepath, encoding='utf-8')
#         if isinstance(text, bytes):
#             text = text.decode('utf-8', errors='ignore')
#         return text
#     except Exception as e:
#         print(f"  ⚠️ textract 失败 {os.path.basename(filepath)}: {str(e)[:100]}")
#         return ""

# def extract_text_pdf_precise(filepath):
#     """优先用 pdfplumber 提取 PDF（更可靠）"""
#     pages = []
#     try:
#         with pdfplumber.open(filepath) as pdf:
#             for i, page in enumerate(pdf.pages):
#                 try:
#                     text = page.extract_text(x_tolerance=1, y_tolerance=1) or ""
#                     if text.strip():
#                         pages.append((i + 1, text))
#                 except:
#                     continue
#     except Exception as e:
#         pass  # fallback to textract
#     return pages

# def classify_sentence(sent):
#     if len(sent) < 15:
#         return 'unclear', None
#     for kw in HUMAN_KEYWORDS:
#         if kw in sent:
#             return 'human', kw
#     for pat in PHYSICAL_PATTERNS:
#         if pat.search(sent):
#             return 'physical', pat.pattern
#     return 'unclear', None

# # ================== 主流程 ==================

# all_records = []

# for region_folder in os.listdir(BASE_DIR):
#     region_path = os.path.join(BASE_DIR, region_folder)
#     if not os.path.isdir(region_path):
#         continue
#     region_label = get_region_label(region_folder)
#     print(f"📁 区域: {region_folder} → {region_label}")

#     for city_folder in os.listdir(region_path):
#         city_path = os.path.join(region_path, city_folder)
#         if not os.path.isdir(city_path):
#             continue
#         city_name = city_folder.rstrip('市')
#         print(f"  🏙️ 城市: {city_name}")

#         for filename in os.listdir(city_path):
#             filepath = os.path.join(city_path, filename)
#             _, ext = os.path.splitext(filename)
#             ext = ext.lower()

#             if ext not in ['.doc', '.docx', '.pdf']:
#                 continue

#             # 提取年份
#             year_match = YEAR_PATTERN.search(filename)
#             if not year_match:
#                 continue
#             year = int(year_match.group(1))
#             file_type = "政府工作报告" if "工作" in filename else "财政预决算报告"

#             sentences_with_pages = []

#             if ext == '.pdf':
#                 # 优先用 pdfplumber
#                 pages = extract_text_pdf_precise(filepath)
#                 if pages:
#                     for page_num, text in pages:
#                         sents = [s.strip() for s in re.split(r'[。；！\n]', text) if len(s) > 15]
#                         for s in sents:
#                             sentences_with_pages.append((page_num, s))
#                 else:
#                     # fallback to textract
#                     text = extract_text_textract(filepath)
#                     sents = [s.strip() for s in re.split(r'[。；！\n]', text) if len(s) > 15]
#                     sentences_with_pages = [(None, s) for s in sents]
#             else:
#                 # .doc / .docx
#                 text = extract_text_textract(filepath)
#                 sents = [s.strip() for s in re.split(r'[。；！\n]', text) if len(s) > 15]
#                 sentences_with_pages = [(None, s) for s in sents]

#             # 分类
#             for page_num, sent in sentences_with_pages:
#                 category, matched_kw = classify_sentence(sent)
#                 if category != 'unclear':
#                     all_records.append({
#                         "区域": region_label,
#                         "城市": city_name,
#                         "年份": year,
#                         "文件类型": file_type,
#                         "支出类别": category,
#                         "匹配关键词": matched_kw,
#                         "原文片段": sent[:250].replace('\n', ' '),
#                         "页码": page_num,
#                         "文件名": filename
#                     })

# # 保存结果
# os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
# df = pd.DataFrame(all_records)
# df.to_csv(OUTPUT_CSV, index=False, encoding='utf_8_sig')
# print(f"\n✅ 共提取 {len(df)} 条记录，已保存至: {OUTPUT_CSV}")

In [25]:
# # analyze_all_reports.py
# import os
# import re
# import pandas as pd
# import warnings
# import textract
# import pdfplumber
# from docx import Document

# warnings.filterwarnings("ignore", message="Cannot render horizontal string")

# # ================== 配置 ==================
# BASE_DIR = "./reports"
# OUTPUT_CSV = "./output/investment_analysis.csv"

# # 区域映射（模糊匹配）
# def get_region_label(name):
#     name = name.strip()
#     if "东部" in name: return "东部"
#     if "中部" in name: return "中部"
#     if "西部" in name: return "西部"
#     return "未知区域"

# # === 关键词定义 ===
# HUMAN_KEYWORDS = [
#     '义务教育', '高中', '职业教育', '职教', '高等教育', '教师', '学生资助',
#     '课后服务', '产教融合', '教育信息化', '特岗教师', '营养午餐',
#     '公共卫生', '疫情防控', '基层医疗', '医保', '妇幼', '精神卫生',
#     '家庭医生', '健康促进', '中医药', '疾控',
#     '养老金', '低保', '特困', '残疾人', '托育', '儿童福利', '就业',
#     '创业扶持', '公益性岗位', '稳岗补贴', '技能培训',
#     '人才引进', '安家费', '租房补贴', '科研启动', '高层次人才', '青年',
#     '技能人才', '工匠',
#     '公共文化', '社区治理', '法律援助', '志愿服务'
# ]

# PHYSICAL_PATTERNS = []
# for kw in [
#     '交通', '道路', '桥梁', '轨道交通', '地铁', '机场', '港口', '水利', '电网',
#     '5G', '数据中心', '新基建', '智慧城市', '地下管廊', '停车场', '充电桩',
#     '产业园', '开发区', '高新区', '工业园', '标准厂房', '物流', '冷链',
#     '孵化器', '平台公司', '土地收储',
#     r'新建.*学校', r'新建.*医院', r'文体场馆', r'公园', r'城市更新', r'棚改',
#     r'安置房', r'保障房'
# ]:
#     if '.*' in kw:
#         PHYSICAL_PATTERNS.append(re.compile(kw))
#     else:
#         PHYSICAL_PATTERNS.append(re.compile(re.escape(kw)))

# YEAR_PATTERN = re.compile(r'(202[1-5])')

# # === 表格识别：是否为目标支出表 ===
# def is_target_table(table):
#     if not table or len(table) < 3:
#         return False
#     # 检查前两行标题
#     header_text = " ".join(str(cell) for row in table[:2] for cell in row if cell)
#     if not re.search(r'(支出.*决算|功能分类|一般公共预算)', header_text, re.IGNORECASE):
#         return False
#     # 检查是否有典型支出科目
#     for row in table[2:]:
#         if not row or not row[0]:
#             continue
#         first_col = str(row[0]).strip()
#         if any(kw in first_col for kw in ['教育', '社会保障', '卫生健康', '城乡社区', '农林水', '交通运输', '住房保障']) and ('支出' in first_col or '合计' in first_col):
#             return True
#     return False

# # === 提取PDF中的目标表格 ===
# def extract_target_tables_from_pdf(pdf_path):
#     tables_found = []
#     try:
#         with pdfplumber.open(pdf_path) as pdf:
#             for page_num, page in enumerate(pdf.pages, 1):
#                 tables = page.extract_tables()
#                 for table in tables:
#                     if is_target_table(table):
#                         tables_found.append((page_num, table))
#     except Exception as e:
#         pass
#     return tables_found

# # === 提取DOCX中的目标表格 ===
# def extract_target_tables_from_docx(docx_path):
#     try:
#         doc = Document(docx_path)
#         tables_found = []
#         for table in doc.tables:
#             data = []
#             for row in table.rows:
#                 data.append([cell.text.strip() for cell in row.cells])
#             if is_target_table(data):
#                 tables_found.append((None, data))  # Word无页码
#         return tables_found
#     except:
#         return []

# # === 处理支出表格（通用）===
# def process_expense_table(table_data, city, year, region, filename, page=None):
#     records = []
#     df = pd.DataFrame(table_data[1:], columns=table_data[0])
#     # 尝试找到“项目”列和“金额”列
#     project_col = None
#     amount_col = None
#     for col in df.columns:
#         col_str = str(col).lower()
#         if '项目' in col_str or '科目' in col_str or '功能' in col_str:
#             project_col = col
#         if '决算' in col_str or '支出' in col_str or '金额' in col_str or '数' in col_str:
#             amount_col = col
#     if project_col is None or amount_col is None:
#         return records

#     for _, row in df.iterrows():
#         project = str(row[project_col]).strip()
#         if not project or project in ['合计', '总计', '小计']:
#             continue
#         raw_amount = str(row[amount_col])
#         # 清理数字：移除非数字字符（保留 . 和 -）
#         clean_amount = re.sub(r'[^\d.\-]', '', raw_amount)
#         try:
#             amount = float(clean_amount) if clean_amount else 0.0
#         except:
#             continue

#         # 分类
#         category = None
#         if any(kw in project for kw in ['教育']):
#             category = 'human'
#         elif any(kw in project for kw in ['卫生', '健康', '医疗']):
#             category = 'human'
#         elif any(kw in project for kw in ['社保', '养老', '就业', '救助', '残疾人', '托育']):
#             category = 'human'
#         elif any(kw in project for kw in ['人才', '引进', '补贴']):
#             category = 'human'
#         elif any(kw in project for kw in ['文化', '旅游', '宣传']):
#             category = 'human'
#         else:
#             category = 'physical'

#         records.append({
#             "区域": region,
#             "城市": city,
#             "年份": year,
#             "文件类型": "财政决算报告",
#             "支出类别": category,
#             "匹配关键词": project,
#             "原文片段": f"{project}: {raw_amount}",
#             "金额（万元）": amount,
#             "页码": page,
#             "文件名": filename
#         })
#     return records

# # === 全文句子分类 ===
# def classify_sentence(sent):
#     if len(sent) < 15:
#         return 'unclear', None
#     for kw in HUMAN_KEYWORDS:
#         if kw in sent:
#             return 'human', kw
#     for pat in PHYSICAL_PATTERNS:
#         if pat.search(sent):
#             return 'physical', pat.pattern
#     return 'unclear', None

# # === 主流程 ===
# all_records = []

# for region_folder in os.listdir(BASE_DIR):
#     region_path = os.path.join(BASE_DIR, region_folder)
#     if not os.path.isdir(region_path): continue
#     region_label = get_region_label(region_folder)
#     print(f"📁 区域: {region_folder} → {region_label}")

#     for city_folder in os.listdir(region_path):
#         city_path = os.path.join(region_path, city_folder)
#         if not os.path.isdir(city_path): continue
#         city_name = city_folder.rstrip('市')
#         print(f"  🏙️ 城市: {city_name}")

#         for filename in os.listdir(city_path):
#             filepath = os.path.join(city_path, filename)
#             _, ext = os.path.splitext(filename)
#             ext = ext.lower()

#             if ext not in ['.doc', '.docx', '.pdf']:
#                 continue

#             year_match = YEAR_PATTERN.search(filename)
#             if not year_match: continue
#             year = int(year_match.group(1))

#             if "工作" in filename:
#                 file_type = "政府工作报告"
#             elif "决算" in filename or "预算" in filename:
#                 file_type = "财政预决算报告"
#             else:
#                 continue

#             # === 处理财政预决算报告（优先表格）===
#             if file_type == "财政预决算报告":
#                 tables = []
#                 if ext == '.pdf':
#                     tables = extract_target_tables_from_pdf(filepath)
#                 elif ext == '.docx':
#                     tables = extract_target_tables_from_docx(filepath)

#                 if tables:
#                     for page_num, table in tables:
#                         records = process_expense_table(
#                             table, city_name, year, region_label, filename, page_num
#                         )
#                         all_records.extend(records)
#                     continue  # 表格已处理，跳过全文

#             # === 处理全文（工作报告 或 无表格的预决算）===
#             text = ""
#             try:
#                 if ext == '.pdf':
#                     with pdfplumber.open(filepath) as pdf:
#                         for page in pdf.pages:
#                             txt = page.extract_text(x_tolerance=1, y_tolerance=1) or ""
#                             text += txt + "\n"
#                 else:  # .doc / .docx
#                     raw = textract.process(filepath, encoding='utf-8')
#                     text = raw.decode('utf-8', errors='ignore') if isinstance(raw, bytes) else str(raw)
#             except Exception as e:
#                 print(f"  ⚠️ 文本提取失败 {filename}: {e}")
#                 continue

#             sentences = [s.strip() for s in re.split(r'[。；！\n]', text) if len(s) > 15]
#             for sent in sentences:
#                 cat, kw = classify_sentence(sent)
#                 if cat != 'unclear':
#                     all_records.append({
#                         "区域": region_label,
#                         "城市": city_name,
#                         "年份": year,
#                         "文件类型": file_type,
#                         "支出类别": cat,
#                         "匹配关键词": kw,
#                         "原文片段": sent[:250].replace('\n', ' '),
#                         "金额（万元）": None,
#                         "页码": None,
#                         "文件名": filename
#                     })

# # === 保存结果 ===
# os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
# df = pd.DataFrame(all_records)
# df.to_csv(OUTPUT_CSV, index=False, encoding='utf_8_sig')
# print(f"\n✅ 共提取 {len(df)} 条记录，结果已保存至: {OUTPUT_CSV}")

In [26]:
import os

# Apple Silicon (M1/M2/M3)
os.environ["PATH"] = "/opt/homebrew/bin:" + os.environ.get("PATH", "")

# 如果是 Intel Mac，用这一行代替：
# os.environ["PATH"] = "/usr/local/bin:" + os.environ.get("PATH", "")

# 验证
!which antiword

/opt/homebrew/bin/antiword


In [ ]:


import os
import re
import pandas as pd
import warnings
import textract
import pdfplumber
from docx import Document

warnings.filterwarnings("ignore", message="Cannot render horizontal string")

# ================== 配置 ==================
BASE_DIR = "./reports"
OUTPUT_CSV = "./output/investment_analysis.csv"

# 区域映射（模糊匹配）
def get_region_label(name):
    name = name.strip()
    if "东部" in name: return "东部"
    if "中部" in name: return "中部"
    if "西部" in name: return "西部"
    return "未知区域"

# === 关键词定义 ===
HUMAN_KEYWORDS = [
    '义务教育', '高中', '职业教育', '职教', '高等教育', '教师', '学生资助',
    '课后服务', '产教融合', '教育信息化', '特岗教师', '营养午餐',
    '公共卫生', '疫情防控', '基层医疗', '医保', '妇幼', '精神卫生',
    '家庭医生', '健康促进', '中医药', '疾控',
    '养老金', '低保', '特困', '残疾人', '托育', '儿童福利', '就业',
    '创业扶持', '公益性岗位', '稳岗补贴', '技能培训',
    '人才引进', '安家费', '租房补贴', '科研启动', '高层次人才', '青年',
    '技能人才', '工匠',
    '公共文化', '社区治理', '法律援助', '志愿服务'
]

PHYSICAL_PATTERNS = []
for kw in [
    '交通', '道路', '桥梁', '轨道交通', '地铁', '机场', '港口', '水利', '电网',
    '5G', '数据中心', '新基建', '智慧城市', '地下管廊', '停车场', '充电桩',
    '产业园', '开发区', '高新区', '工业园', '标准厂房', '物流', '冷链',
    '孵化器', '平台公司', '土地收储',
    r'新建.*学校', r'新建.*医院', r'文体场馆', r'公园', r'城市更新', r'棚改',
    r'安置房', r'保障房'
]:
    if '.*' in kw:
        PHYSICAL_PATTERNS.append(re.compile(kw))
    else:
        PHYSICAL_PATTERNS.append(re.compile(re.escape(kw)))

YEAR_PATTERN = re.compile(r'(202[1-5])')

# === 表格识别：是否为目标支出表 ===
def is_target_table(table):
    if not table or len(table) < 3:
        return False
    # 检查前两行标题
    header_text = " ".join(str(cell) for row in table[:2] for cell in row if cell)
    if not re.search(r'(支出.*决算|功能分类|一般公共预算)', header_text, re.IGNORECASE):
        return False
    # 检查是否有典型支出科目
    for row in table[2:]:
        if not row or not row[0]:
            continue
        first_col = str(row[0]).strip()
        if any(kw in first_col for kw in ['教育', '社会保障', '卫生健康', '城乡社区', '农林水', '交通运输', '住房保障']) and ('支出' in first_col or '合计' in first_col):
            return True
    return False

# === 提取PDF中的目标表格 ===
def extract_target_tables_from_pdf(pdf_path):
    tables_found = []
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 1):
                tables = page.extract_tables()
                for table in tables:
                    if is_target_table(table):
                        tables_found.append((page_num, table))
    except Exception as e:
        pass
    return tables_found

# === 提取DOCX中的目标表格 ===
def extract_target_tables_from_docx(docx_path):
    try:
        doc = Document(docx_path)
        tables_found = []
        for table in doc.tables:
            data = []
            for row in table.rows:
                data.append([cell.text.strip() for cell in row.cells])
            if is_target_table(data):
                tables_found.append((None, data))  # Word无页码
        return tables_found
    except:
        return []

# === 处理支出表格（通用）===
def process_expense_table(table_data, city, year, region, filename, page=None):
    records = []
    df = pd.DataFrame(table_data[1:], columns=table_data[0])
    # 尝试找到“项目”列和“金额”列
    project_col = None
    amount_col = None
    for col in df.columns:
        col_str = str(col).lower()
        if '项目' in col_str or '科目' in col_str or '功能' in col_str:
            project_col = col
        if '决算' in col_str or '支出' in col_str or '金额' in col_str or '数' in col_str:
            amount_col = col
    if project_col is None or amount_col is None:
        return records

    for _, row in df.iterrows():
        project = str(row[project_col]).strip()
        if not project or project in ['合计', '总计', '小计']:
            continue
        raw_amount = str(row[amount_col])
        # 清理数字：移除非数字字符（保留 . 和 -）
        clean_amount = re.sub(r'[^\d.\-]', '', raw_amount)
        try:
            amount = float(clean_amount) if clean_amount else 0.0
        except:
            continue

        # 分类
        category = None
        if any(kw in project for kw in ['教育']):
            category = 'human'
        elif any(kw in project for kw in ['卫生', '健康', '医疗']):
            category = 'human'
        elif any(kw in project for kw in ['社保', '养老', '就业', '救助', '残疾人', '托育']):
            category = 'human'
        elif any(kw in project for kw in ['人才', '引进', '补贴']):
            category = 'human'
        elif any(kw in project for kw in ['文化', '旅游', '宣传']):
            category = 'human'
        else:
            category = 'physical'

        records.append({
            "区域": region,
            "城市": city,
            "年份": year,
            "文件类型": "财政决算报告",
            "支出类别": category,
            "匹配关键词": project,
            "原文片段": f"{project}: {raw_amount}",
            "金额（万元）": amount,
            "页码": page,
            "文件名": filename
        })
    return records

# === 全文句子分类 ===
def classify_sentence(sent):
    if len(sent) < 15:
        return 'unclear', None
    for kw in HUMAN_KEYWORDS:
        if kw in sent:
            return 'human', kw
    for pat in PHYSICAL_PATTERNS:
        if pat.search(sent):
            return 'physical', pat.pattern
    return 'unclear', None

# === 主流程 ===# === 主流程（带实时写入）===
all_records = []  # 全局累计

# 获取所有 (region, city) 对，便于排序和追踪
city_list = []
for region_folder in os.listdir(BASE_DIR):
    region_path = os.path.join(BASE_DIR, region_folder)
    if not os.path.isdir(region_path): continue
    for city_folder in os.listdir(region_path):
        city_path = os.path.join(region_path, city_folder)
        if os.path.isdir(city_path):
            city_list.append((region_folder, city_folder))

total_cities = len(city_list)
print(f"📊 共发现 {total_cities} 个城市，开始处理...\n")

for idx, (region_folder, city_folder) in enumerate(city_list, 1):
    region_path = os.path.join(BASE_DIR, region_folder)
    city_path = os.path.join(region_path, city_folder)

    region_label = get_region_label(region_folder)
    city_name = city_folder.rstrip('市')

    print(f"[{idx}/{total_cities}] 🏙️ 正在处理: {city_name} ({region_label})")

    city_records = []

    for filename in os.listdir(city_path):
        # 跳过隐藏文件（以 . 开头）和临时文件
        if filename.startswith('.'):
            continue
        filepath = os.path.join(city_path, filename)
        _, ext = os.path.splitext(filename)
        ext = ext.lower()

        if ext not in ['.doc', '.docx', '.pdf']:
            continue

        year_match = YEAR_PATTERN.search(filename)
        if not year_match: continue
        year = int(year_match.group(1))

        if "工作" in filename:
            file_type = "政府工作报告"
        elif "决算" in filename or "预算" in filename:
            file_type = "财政预决算报告"
        else:
            continue

        # === 处理财政预决算报告（优先表格）===
        if file_type == "财政预决算报告":
            tables = []
            try:
                if ext == '.pdf':
                    tables = extract_target_tables_from_pdf(filepath)
                elif ext == '.docx':
                    # 额外检查：确保是有效 DOCX（非临时文件）
                    if os.path.getsize(filepath) < 1024:  # 小于1KB 视为无效
                        print(f"  ⚠️ 跳过空文件 {filename}")
                        continue
                    tables = extract_target_tables_from_docx(filepath)
            except Exception as e:
                print(f"  ⚠️ 表格提取失败 {filename}: {e}")
                tables = []
            if tables:
                for page_num, table in tables:
                    records = process_expense_table(
                        table, city_name, year, region_label, filename, page_num
                    )
                    city_records.extend(records)
                continue

        # === 处理全文 ===
        text = ""
        try:
            if ext == '.pdf':
                with pdfplumber.open(filepath) as pdf:
                    for page in pdf.pages:
                        txt = page.extract_text(x_tolerance=1, y_tolerance=1) or ""
                        text += txt + "\n"
            else:
                raw = textract.process(filepath, encoding='utf-8')
                text = raw.decode('utf-8', errors='ignore') if isinstance(raw, bytes) else str(raw)
        except Exception as e:
             # 不再打印完整错误，只提示文件名
            print(f"  ⚠️ 跳过无法读取的文件: {filename}")            
            continue

        sentences = [s.strip() for s in re.split(r'[。；！\n]', text) if len(s) > 15]
        for sent in sentences:
            cat, kw = classify_sentence(sent)
            if cat != 'unclear' and "收入" not in sent:
                city_records.append({
                    "区域": region_label,
                    "城市": city_name,
                    "年份": year,
                    "文件类型": file_type,
                    "支出类别": cat,
                    "匹配关键词": kw,
                    "原文片段": sent[:250].replace('\n', ' '),
                    "金额（万元）": None,
                    "页码": None,
                    "文件名": filename
                })

    # 将当前城市结果加入全局
    all_records.extend(city_records)
    print(f"  ✅ 完成 {city_name}，新增 {len(city_records)} 条记录，累计 {len(all_records)} 条")

    # === 实时写入 CSV（覆盖模式）===
    os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
    df = pd.DataFrame(all_records)
    df.to_csv(OUTPUT_CSV, index=False, encoding='utf_8_sig')
    print(f"  💾 已保存至: {OUTPUT_CSV}\n")

📊 共发现 15 个城市，开始处理...

[1/15] 🏙️ 正在处理: 咸阳 (西部)
  ✅ 完成 咸阳，新增 371 条记录，累计 371 条
  💾 已保存至: ./output/investment_analysis.csv

[2/15] 🏙️ 正在处理: 自贡 (西部)
  ✅ 完成 自贡，新增 463 条记录，累计 834 条
  💾 已保存至: ./output/investment_analysis.csv

[3/15] 🏙️ 正在处理: 西安 (西部)
  ✅ 完成 西安，新增 1364 条记录，累计 2198 条
  💾 已保存至: ./output/investment_analysis.csv

[4/15] 🏙️ 正在处理: 成都 (西部)
  ✅ 完成 成都，新增 856 条记录，累计 3054 条
  💾 已保存至: ./output/investment_analysis.csv

[5/15] 🏙️ 正在处理: 南宁 (西部)
  ✅ 完成 南宁，新增 893 条记录，累计 3947 条
  💾 已保存至: ./output/investment_analysis.csv

[6/15] 🏙️ 正在处理: 上海 (东部)
  ✅ 完成 上海，新增 620 条记录，累计 4567 条
  💾 已保存至: ./output/investment_analysis.csv

[7/15] 🏙️ 正在处理: 苏州 (东部)
  ✅ 完成 苏州，新增 390 条记录，累计 4957 条
  💾 已保存至: ./output/investment_analysis.csv

[8/15] 🏙️ 正在处理: 深圳 (东部)
  ⚠️ 跳过无法读取的文件: 深圳市2021预算.doc
  ✅ 完成 深圳，新增 455 条记录，累计 5412 条
  💾 已保存至: ./output/investment_analysis.csv

[9/15] 🏙️ 正在处理: 杭州 (东部)


Cannot render horizontal string because /b'cm' is not a valid int, float or bytes.
Cannot render horizontal string because /b'cm' is not a valid int, float or bytes.
Cannot render horizontal string because /b'BT' is not a valid int, float or bytes.
Cannot render horizontal string because /b'cm' is not a valid int, float or bytes.
Cannot render horizontal string because /b'BT' is not a valid int, float or bytes.
Cannot render horizontal string because /b'Tz' is not a valid int, float or bytes.
Cannot render horizontal string because /b'Tm' is not a valid int, float or bytes.
Cannot render horizontal string because [b'2', 0.276, /b'Q', /b'q', /'F5', 14.749, /b'Tf', /b'q', 0, /b'g', /b'q', 1, 0, 0, -1, 0, 0, /b'cm', 1, 0, 0, 51, 1, 0, 0, -1, 0, 0, /b'cm', 1, 0, 0, 1, 300.615, -176.2, /b'Tm3', 1, 0, 0, -1, 0, 0, /b'cm', 1, 0, 0, 1, 266.07, -206.296, /b'cm', /b'BT', 11, 0, 0, -1, 0, 0, /b'cm', 1, 0, 0, 1, 141.768, -206.296, /b'cm', /b'BT', 100, /b'Tz', 1, 0, 0, 1, 0, -34, /b'T', /b'Q', 1, 0

  ⚠️ 跳过无法读取的文件: 杭州市2023年预算.pdf


Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value


  ✅ 完成 杭州，新增 1076 条记录，累计 6488 条
  💾 已保存至: ./output/investment_analysis.csv

[10/15] 🏙️ 正在处理: 北京 (东部)
  ✅ 完成 北京，新增 695 条记录，累计 7183 条
  💾 已保存至: ./output/investment_analysis.csv

[11/15] 🏙️ 正在处理: 长沙 (中部)
  ⚠️ 跳过无法读取的文件: 2长沙市工作报告2022.docx
  ⚠️ 跳过无法读取的文件: 4长沙市2024预算.docx
  ⚠️ 跳过无法读取的文件: 1长沙市2021预算.docx
  ⚠️ 跳过无法读取的文件: 5长沙市2025预算.docx
  ⚠️ 跳过无法读取的文件: 5长沙市工作报告2025.docx
  ⚠️ 跳过无法读取的文件: 1长沙市工作报告2021.docx
  ⚠️ 跳过无法读取的文件: 2长沙市2022预算.docx
  ⚠️ 跳过无法读取的文件: 3长沙市2023预算.docx
  ✅ 完成 长沙，新增 129 条记录，累计 7312 条
  💾 已保存至: ./output/investment_analysis.csv

[12/15] 🏙️ 正在处理: 合肥 (中部)
  ⚠️ 跳过无法读取的文件: 5合肥市工作报告2025.docx
  ✅ 完成 合肥，新增 532 条记录，累计 7844 条
  💾 已保存至: ./output/investment_analysis.csv

[13/15] 🏙️ 正在处理: 武汉 (中部)
  ⚠️ 跳过无法读取的文件: 5武汉市工作报告2025.docx
  ✅ 完成 武汉，新增 248 条记录，累计 8092 条
  💾 已保存至: ./output/investment_analysis.csv

[14/15] 🏙️ 正在处理: 南昌 (中部)
  ✅ 完成 南昌，新增 374 条记录，累计 8466 条
  💾 已保存至: ./output/investment_analysis.csv

[15/15] 🏙️ 正在处理: 郑州 (中部)
  ⚠️ 跳过无法读取的文件: 5郑州市工作报告2025.docx


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBB

  ✅ 完成 郑州，新增 530 条记录，累计 8996 条
  💾 已保存至: ./output/investment_analysis.csv

